<a href="https://colab.research.google.com/github/TillVollmer5/DSF-FS2022-Till-Vollmer/blob/main/CodingTask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Science Fundamentals for DCBP, S. Haug, University of Bern 

# Coding Task 2

- Deadline (upload to Ilias): 2022-05-15 24:00
- Solutions in lecture on 2022-05-18
- Estimated effort: from 4 to 60 hours
- Images (data) on Ilias in the Data folder, the images.zip file
- Main notebook is number 11 for this task, however, also others may help you. You can also benefit a lot from googeling for code examples


## Subtasks (maximum 12.5 points)

- Upload the images to a folder on your google drive. Write a function which retrieves the filenames of the images, contains a loop converting them to png images and saves them. [about 1 hour] [max 3 points] 
- With some filtering and sthresholding, write a function which converts the colored bins into nice white segments on a black background for one image. [about 1 hour][max 3 points]
- Write an algorithm which counts the number of bins. First approach could be a "sliding window" algorithm, i.e a loop sliding over the image and counting regions with substantial white content. There may also be nice segmentation tools out "there" already. You may google. [about 4 hours][max 3 points] 
- Write an algorithm which does the latter task on all images. As the images are quite different, this is a large task which goes beyond the time budget of this exercise, however, you can give it a try. If it works out for a couple of images, that is already good. [from 4 to 50 hours][max 3.5 points]

Work in teams and have fun ! 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.filters as skf
import skimage.morphology as skm
!pip install pillow_heif
from PIL import Image
import pillow_heif

     |████████████████████████████████| 13.1 MB 7.1 MB/s 


# Part 1

Import of images from google drive and conversion to png

In [12]:
from google.colab import drive
drive.mount('/content/drive')

for i in range(5379, 5393, 1):
  url_pre = '/content/drive/MyDrive/DSF/Images/'
  url_im_name = ''.join(['IMG_', str(i), '.HEIC'])

  print(url_im_name)
  heif_file = pillow_heif.read(url_pre+url_im_name)
  image = Image.frombytes(heif_file.mode,heif_file.size,heif_file.data,"raw",)
  url_im_name = url_im_name.split('.')[0]+'.png'
  image.save(url_pre+url_im_name, format="png")
 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
IMG_5379.HEIC
IMG_5380.HEIC
IMG_5381.HEIC
IMG_5382.HEIC
IMG_5383.HEIC
IMG_5384.HEIC
IMG_5385.HEIC
IMG_5386.HEIC
IMG_5387.HEIC
IMG_5388.HEIC
IMG_5389.HEIC
IMG_5390.HEIC
IMG_5391.HEIC
IMG_5392.HEIC


# Part 2

Image processing as preparation for the Part 3 with spot counting.

In [17]:
for i in range(5379, 5393, 1):
  url_pre = '/content/drive/MyDrive/DSF/Images/'
  url_name = ''.join(['IMG_', str(i), '.HEIC'])
  imageRGB  = io.imread(url_pre+url_name)
  plt.imshow(imageRGB)
  image = imageRGB[:,:,2]
  #resizing
  image = image[image.shape[0]//2:image.shape[0],:]
  print(image.shape)
  plt.imshow(image, cmap ='gray')
  plt.show()
  #filtering

  #thresholding
  threshold_manual = 60
  image_threshold = image_mean<threshold_manual
  plt.figure(figsize=(12, 12))
  plt.imshow(image_threshold, cmap ='gray')
  plt.show()

ValueError: ignored